<a href="https://colab.research.google.com/github/AnshuTechie/Sentiment-Analysis-and-Face-Recognition/blob/main/Sentiment_Analysis_%26_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

MessageError: Error: credential propagation was unsuccessful

Source : https://github.com/aishwaryakore5696/Face-Recognition-and-Emotion-recognition-for-Sentiment-Analysis/blob/main/FaceRecognitionEmotionDetection.ipynb

In [ ]:
! pip install mtcnn

In [ ]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

In [ ]:
# demonstrate face detection on 5 Celebrity Faces Dataset

from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# specify folder to plot
folder = './gdrive/My Drive/facenet/images/5-celebrity-faces-dataset/train/ben_afflek/'
i = 1
# enumerate files
for filename in listdir(folder):
	# path
	path = folder + filename
	# get face
	face = extract_face(path,(160, 160))
	print(i, face.shape)
	# plot
	pyplot.subplot(2, 7, i)
	pyplot.axis('off')
	pyplot.imshow(face)
	i += 1
pyplot.show()


In [ ]:

# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path,(160, 160))
		# store
		faces.append(face)
	return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)


#please replace the path mentioned below with the path of your dataset.
# load train dataset
trainX, trainy = load_dataset('./gdrive/My Drive/facenet/images/5-celebrity-faces-dataset/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('./gdrive/My Drive/facenet/images/5-celebrity-faces-dataset/val/')
# save arrays to one file in compressed format
savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

In [ ]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

# load the face dataset
data = load('5-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
modelem = load_model('./gdrive/My Drive/facenet/model/facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(modelem, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(modelem, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [ ]:
# load dataset
data = load('5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load testing data faces
data2 = load('5-celebrity-faces-dataset.npz')
testX_faces = data2['arr_2']
# load training dataface embeddings
data = load('5-celebrity-faces-embeddings.npz')
trainX, trainy, testXem, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testXem)
#print("test embeddings")
#print(testXem)
#print(testXem[0].shape)
#print("test")
#print(testX)
#print(testX.shape)
#print(testX[0].shape)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
modelsvc = SVC(kernel='linear', probability=True)
modelsvc.fit(trainX, trainy)
# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
#print("random_face_emb",testX[selection])
#print(testX[selection].shape)
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
#print("samples:",samples)
#print(samples.shape)
yhat_class = modelsvc.predict(samples)
yhat_prob = modelsvc.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])
# plot for fun
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()
print("Predicted ",predict_names[0]," with probability ",class_probability)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import keras
from keras.preprocessing import image

from keras import applications as keras_applications

!pip install keras_applications
!pip install git+https://github.com/rcmalli/keras-vggface.git

from keras_vggface.vggface import VGGFace
from keras_vggface import utils


In [ ]:

import keras
from keras.preprocessing import image
from keras import applications as keras_applications
from keras_vggface.vggface import VGGFace
from keras_vggface import utils


In [ ]:
# os.system('tar -xf /kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz');
#please replace the path mentioned below with the path of your fer2013 dataset.
data = pd.read_csv('./gdrive/My Drive/facenet/fer2013/fer2013.csv')
# ./gdrive/My Drive/facenet/images/5-celebrity-faces-dataset/train/
data

In [ ]:

data.Usage.unique()
train_data = data[data.Usage=='Training']
val_data = data[data.Usage=='PublicTest']
test_data = data[data.Usage=='PrivateTest']
train_data.shape, val_data.shape, test_data.shape

In [ ]:
!pip install imblearn
import collections
import imblearn
from imblearn import under_sampling, over_sampling
oversampler = imblearn.over_sampling.RandomOverSampler()

In [ ]:
collections.Counter(train_data.emotion)

In [ ]:

from keras.utils import to_categorical

x_train, y_train = oversampler.fit_resample(train_data.pixels.values.reshape(-1,1),train_data.emotion.values)

# x_train = train_data.pixels.values.reshape(-1,1)
# y_train = train_data.emotion.values

x_val = val_data.pixels.values.reshape(-1,1)
y_val = val_data.emotion.values

x_test = test_data.pixels.values.reshape(-1,1)
y_test = test_data.emotion.values


In [ ]:
collections.Counter(y_train)

In [ ]:

y_train = to_categorical(y_train,num_classes=7)
y_val   = to_categorical(y_val  ,num_classes=7)
y_test  = to_categorical(y_test ,num_classes=7)

In [ ]:
from copy import deepcopy as copy

def smooth_labels(y, smooth_factor):
    '''Convert a matrix of one-hot row-vector labels into smoothed versions.

    # Arguments
        y: matrix of one-hot row-vector labels to be smoothed
        smooth_factor: label smoothing factor (between 0 and 1)

    # Returns
        A matrix of smoothed labels.
    '''
    assert len(y.shape) == 2, 'input should be a batch of one-hot-encoded data'
    y2 = copy(y)
    if 0 <= smooth_factor <= 1:
        # label smoothing ref: https://www.robots.ox.ac.uk/~vgg/rg/papers/reinception.pdf
        y2 *= 1 - smooth_factor
        y2 += smooth_factor / y.shape[1]
    else:
        raise Exception(
            'Invalid label smoothing factor: ' + str(smooth_factor))
    return y2

In [ ]:

from keras.utils import Sequence
from keras.utils import to_categorical
import cv2
from math import floor

class data_sequence(Sequence):
    '''
      yield sequence of data
      features -- list of features
      labels -- list of labels
      target_channels {int} -- 1 (gray) or 3(RGB)
    '''
    def __init__(self, features, labels, batch_size=128, target_dim=(224,224),
                 n_classes=7, shuffle=True, smooth=0.0):
        'Initialization'
        assert len(features)==len(labels), 'number of feature and labels not consistent'
        self.features = features
        self.labels = labels
        self.batch_size = batch_size
        self.target_dim = target_dim
        self.target_channels = 3
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.smooth = smooth
        self.sample_count = len(labels)
        self.indexes = np.arange(self.sample_count)
        self.on_epoch_end()
#         self.verbose = verbose

    def __len__(self):
        'Denotes the number of batches per epoch'
        return floor(self.sample_count / self.batch_size)

    def __gray2RGB__(self,x):
      if len(x.shape)==2:
        return np.stack((x,x,x),-1)
      else:
        assert len(x.shape)==3
        if len(x[0,0,:]) == 1:
          return np.stack((x[:,:,0],x[:,:,0],x[:,:,0]),-1)
        else:
          assert len(x[0,0,:])==self.target_channels
      return x


    def __getitem__(self, idx):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, *self.target_dim, self.target_channels))
        Y = np.empty((self.batch_size, self.n_classes))
        for i,ind in enumerate(indexes):
          x = self.features[ind]
          # resize image to the target size
          x = cv2.resize(x,self.target_dim,interpolation=cv2.INTER_CUBIC)
          x = self.__gray2RGB__(x)
          X[i] = utils.preprocess_input(x, version=2) # or version=2 for VGGFace2 ResNet50
          y = self.labels[ind]
          if isinstance(y,int):
            Y[i]=to_categorical(y,7)
          else:
            assert len(y)==self.n_classes
            Y[i]=y
        X = np.array(X)
        Y = np.array(Y)
        if self.smooth > 0.0:
          smooth_labels(Y, self.smooth)
        return X,Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [ ]:
x_train = list(x_train)
x_val   = list(x_val)
x_test  = list(x_test)

for i,item in enumerate(x_train):
    x_train[i] = np.fromstring(item[0],sep=' ').reshape(48,48,1)
for i,item in enumerate(x_val):
    x_val[i] = np.fromstring(item[0],sep=' ').reshape(48,48,1)
for i,item in enumerate(x_test):
    x_test[i] = np.fromstring(item[0],sep=' ').reshape(48,48,1)

x_train = np.vstack(x_train).reshape(-1,48,48,1)
x_val = np.vstack(x_val).reshape(-1,48,48,1)
x_test = np.vstack(x_test).reshape(-1,48,48,1)

In [ ]:
train_sequence = data_sequence(x_train,y_train,batch_size=16,target_dim=(224,224),n_classes=7,shuffle=False)
feature,lable = train_sequence.__getitem__(0)
print(train_sequence)

In [ ]:
emotion_dict = {0: 'Angry', 1:'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6:'Neutral'}

import matplotlib.pyplot as plt
plt.imshow(feature[0,:,:,:])
plt.title(emotion_dict[np.argmax(lable[0])]);
print("Predicted emotion",emotion_dict[np.argmax(lable[0])])

In [ ]:
vggface = VGGFace(model='resnet50', include_top=False, input_shape = (224,224,3))
vggface.trainable = False
vggface.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

emotionalmodel = Sequential([vggface,
                    Flatten(),
                    Dropout(0.25),
                    Dense(2048, activation='relu'),
                    Dropout(0.25),
                    Dense(1024, activation='relu'),
                    Dense(7, activation='softmax', name = 'classifer')])
emotionalmodel.summary()

In [ ]:
train_sequence = data_sequence(x_train,y_train,batch_size=64,target_dim=(224,224),n_classes=7,shuffle=True,smooth=0.07)
val_sequence   = data_sequence(x_val,  y_val,  batch_size=64,target_dim=(224,224),n_classes=7,shuffle=True,smooth=0.0)
test_sequence  = data_sequence(x_test, y_test, batch_size=64,target_dim=(224,224),n_classes=7,shuffle=True,smooth=0.0)


In [ ]:
emotionalmodel.compile(optimizer = keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
hist = emotionalmodel.fit_generator(generator = train_sequence,
                            validation_data = val_sequence,
                            epochs = 5)
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(hist.history['accuracy'], marker='', color='blue', linewidth=3,label="train_accuracy")
plt.plot(hist.history['val_accuracy'], marker='', color='red', linewidth=3,label="val_accuracy")
plt.xlabel('epochs')
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist.history['loss'],marker='', color='blue', linewidth=3,label="train_loss")
plt.plot(hist.history['val_loss'],marker='', color='red', linewidth=3,label="val_loss")
plt.xlabel('epochs')
plt.legend()

In [ ]:
emotionalmodel2 = Sequential([vggface,
                    Flatten(),
                    Dropout(0.25),
                    Dense(2048, activation='relu'),
                    Dropout(0.25),
                    Dense(1024, activation='relu'),
                    Dense(7, activation='softmax', name = 'classifer')])
emotionalmodel2.compile(optimizer = keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
hist2 = emotionalmodel2.fit_generator(generator = train_sequence,
                            validation_data = val_sequence,
                            epochs = 5)
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(hist2.history['accuracy'], marker='', color='blue', linewidth=3,label="train_accuracy")
plt.plot(hist2.history['val_accuracy'], marker='', color='red', linewidth=3,label="val_accuracy")
plt.xlabel('epochs')
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist2.history['loss'],marker='', color='blue', linewidth=3,label="train_loss")
plt.plot(hist2.history['val_loss'],marker='', color='red', linewidth=3,label="val_loss")
plt.xlabel('epochs')
plt.legend()


In [ ]:
emotionalmodel3 = Sequential([vggface,
                    Flatten(),
                    Dropout(0.25),
                    Dense(2048, activation='relu'),
                    Dropout(0.25),
                    Dense(1024, activation='relu'),
                    Dense(7, activation='softmax', name = 'classifer')])
emotionalmodel3.compile(optimizer = keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
hist3 = emotionalmodel3.fit_generator(generator = train_sequence,
                            validation_data = val_sequence,
                            epochs = 5)
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(hist3.history['accuracy'], marker='', color='blue', linewidth=3,label="train_accuracy")
plt.plot(hist3.history['val_accuracy'], marker='', color='red', linewidth=3,label="val_accuracy")
plt.xlabel('epochs')
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist3.history['loss'],marker='', color='blue', linewidth=3,label="train_loss")
plt.plot(hist3.history['val_loss'],marker='', color='red', linewidth=3,label="val_loss")
plt.xlabel('epochs')
plt.legend()

In [ ]:
emotionalmodel4 = Sequential([vggface,
                    Flatten(),
                    Dropout(0.25),
                    Dense(2048, activation='relu'),
                    Dropout(0.25),
                    Dense(1024, activation='relu'),
                    Dense(7, activation='softmax', name = 'classifer')])
emotionalmodel4.compile(optimizer = keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
hist4 = emotionalmodel4.fit_generator(generator = train_sequence,
                            validation_data = val_sequence,
                            epochs = 10)
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(hist4.history['accuracy'], marker='', color='blue', linewidth=3,label="train_accuracy")
plt.plot(hist4.history['val_accuracy'], marker='', color='red', linewidth=3,label="val_accuracy")
plt.xlabel('epochs')
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist4.history['loss'],marker='', color='blue', linewidth=3,label="train_loss")
plt.plot(hist4.history['val_loss'],marker='', color='red', linewidth=3,label="val_loss")
plt.xlabel('epochs')
plt.legend()

In [ ]:
emotionalmodel5 = Sequential([vggface,
                    Flatten(),
                    Dropout(0.25),
                    Dense(2048, activation='relu'),
                    Dropout(0.25),
                    Dense(1024, activation='relu'),
                    Dense(7, activation='softmax', name = 'classifer')])
emotionalmodel5.compile(optimizer = keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
hist5 = emotionalmodel5.fit_generator(generator = train_sequence,
                            validation_data = val_sequence,
                            epochs = 20)
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(hist5.history['accuracy'], marker='', color='blue', linewidth=3,label="train_accuracy")
plt.plot(hist5.history['val_accuracy'], marker='', color='red', linewidth=3,label="val_accuracy")
plt.xlabel('epochs')
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist5.history['loss'],marker='', color='blue', linewidth=3,label="train_loss")
plt.plot(hist5.history['val_loss'],marker='', color='red', linewidth=3,label="val_loss")
plt.xlabel('epochs')
plt.legend()

In [ ]:
plt.figure(figsize=(25,3))
plt.subplot(1,3,1)
plt.plot(hist5.history['val_accuracy'], marker='', color='red', linewidth=3,label="20epoch")
plt.subplot(1,3,1)
plt.plot(hist4.history['val_accuracy'], marker='', color='blue', linewidth=3,label="10epochs")
plt.subplot(1,3,1)
plt.plot(hist2.history['val_accuracy'], marker='', color='green', linewidth=3,label="5epochs")
plt.xlabel('epochs')
plt.ylabel('Validation Accuracy')
plt.legend()

In [ ]:
emotionalmodel4.evaluate(test_sequence)

In [ ]:
feature2,lable2 = test_sequence.__getitem__(0)
emotion_dict = {0: 'Angry', 1:'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6:'Neutral'}

import matplotlib.pyplot as plt
plt.imshow(feature2[4,:,:,:])
print("Expected emotion:",emotion_dict[np.argmax(lable2[4])])
res=emotionalmodel4.predict(feature2)
print("Predicted Emotion:",emotion_dict[np.argmax(res[4])],"with probabiltity:",res[4][np.argmax(res[4])])

In [ ]:
import tensorflow as tf
y_pred = []
y_true = []
for x,y in test_sequence:
    y_pred = y_pred + list(emotionalmodel4.predict_classes(x))
    y_true = y_true + list(np.argmax(y,axis=1))

In [ ]:
confusion_matrix = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
confusion_matrix = np.around(confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis], decimals=2)

confusion_matrix = pd.DataFrame(confusion_matrix,
                                index = emotion_dict.values(),
                                columns = emotion_dict.values())

In [ ]:
import seaborn as sns
figure = plt.figure(figsize=(8, 8))
sns.heatmap(confusion_matrix, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
def pred(image):
  #face recognition
  image2 = cv2.imread(image)
  image2=cv2.cvtColor(image2,cv2.COLOR_BGR2RGB)
#Show the image with matplotlib
  plt.imshow(image2)
  plt.show()
  facearray=extract_face(image,(160, 160))
  embedding2 = get_embedding(modelem, facearray)
  li=[]
  li.append(embedding2)
  in_encoder = Normalizer(norm='l2')
  random_face_emb2 = in_encoder.transform(li)
  samples2 = expand_dims(random_face_emb2[0], axis=0)
  yhat_class2 = modelsvc.predict(samples2)
  yhat_prob2 = modelsvc.predict_proba(samples)
  class_index2 = yhat_class2[0]
  predict_names2 = out_encoder.inverse_transform(yhat_class2)

  #emotion detection
  facearrayem=extract_face(image,(224, 224))
  facearrayem = facearrayem.astype('float32')
  X = utils.preprocess_input(facearrayem, version=2) # or version=2 for VGGFace2 ResNet50
  X = np.array(X)
  ls=np.array([X])
  res2=emotionalmodel4.predict(ls)
  pyplot.imshow(facearray)
  pyplot.show()
  print(predict_names2[0],"is feeling",emotion_dict[np.argmax(res2[0])],"today")
  print("Face Recognition probability:",yhat_prob2[0][np.argmax(yhat_prob2)],"Emotion Detection probability:",res2[0][np.argmax(res2[0])])


In [ ]:
#enter the path of the image that you would want to test for face recognition and  emotion detection
pred('/content/gdrive/MyDrive/facenet/images/5-celebrity-faces-dataset/aishwarya.jpeg')


In [ ]:
pred('/content/gdrive/MyDrive/facenet/images/5-celebrity-faces-dataset/mindyangry.jpg')

In [ ]:
pred('/content/gdrive/MyDrive/facenet/images/5-celebrity-faces-dataset/jerry2.jpg')

In [ ]:

pred('/content/gdrive/MyDrive/facenet/images/5-celebrity-faces-dataset/madonnacry.jpg')